In [1]:
from dance.data.base import Data
import anndata as ad
import numpy as np
import pandas as pd

In [2]:
ref_count=pd.read_csv("./spatial_DWLS_data/count.csv",index_col=0).T
ref_annot=pd.read_csv("./spatial_DWLS_data/annot.csv",index_col=0).rename(columns={"cell_types":"cellType"})

In [3]:

adata_inf=ad.AnnData()
adata_inf = ad.AnnData(
            ref_count.values,
            dtype=np.float32,
            obs=ref_annot,
            var=pd.DataFrame(index=ref_count.columns.tolist()),
        )
adata_ref=ad.AnnData()

adata_ref = ad.AnnData(
            ref_count.values,
            dtype=np.float32,
            obs=ref_annot,
            var=pd.DataFrame(index=ref_count.columns.tolist()),
        )


In [4]:
import numpy as np
def standardise(x, center=True, scale=True):
  if center and scale:
    mean = np.mean(x, axis=0)
    x_centered = x - mean  
    std = np.std(x_centered, axis=0, ddof=1)
    x_scaled = x_centered / std
    return x_scaled

  elif center and not scale:  
    mean = np.mean(x, axis=0)
    return x - mean

  elif not center and scale:
    std = np.std(x, axis=0)
    return x / std

  else:
    return x

In [5]:
import scanpy as sc
import math
sc.pp.normalize_total(adata_ref,target_sum=6000)
adata_ref.X=adata_ref.X+1
sc.pp.log1p(adata_ref)
adata_ref.X=adata_ref.X/math.log(2)
adata_ref.X=np.transpose(standardise(np.transpose(standardise(adata_ref.X))))

In [6]:
adata_ref.X

array([[-0.92007875,  0.15731734,  0.17702308,  0.42443213,  0.14004673,
         0.6161042 , -0.2790136 , -1.2760187 , -1.1774621 , -1.3881428 ,
        -1.3785487 , -1.2756894 ,  0.7487154 ,  1.2803595 ,  0.6243716 ,
        -0.86165065,  1.6135772 ,  1.4742669 ,  0.7397503 ,  0.56064016],
       [-1.3229831 ,  0.84048784,  0.698226  ,  0.64299977,  0.552692  ,
         1.0153862 ,  0.01275463, -1.832667  ,  0.36854628,  0.11915276,
         0.38909566,  0.23673327, -1.8435562 , -1.6988747 ,  0.37529483,
        -1.2393179 ,  1.207723  ,  1.0337595 ,  0.19174537,  0.25280187],
       [-0.78841656,  0.9086625 ,  0.9557844 ,  0.7659593 ,  0.35076457,
         0.9222148 ,  0.78336906, -1.2107875 , -1.0938368 ,  1.0520781 ,
        -1.3324534 ,  1.1362419 ,  1.0874697 , -1.099915  , -1.4814345 ,
        -0.7190837 , -0.8030248 , -0.89807427,  0.76074505,  0.7037367 ],
       [ 1.0951704 ,  0.86379504,  0.12247148,  0.7754688 ,  0.536213  ,
        -1.2587472 ,  0.52245593, -1.2434225 ,  

In [7]:
from dance.transforms.filter import FilterGenesMarkerGini
from dance.transforms.pseudo_gen import CellTopicProfile,CellGiottoTopicProfile,CellTypeNums
from dance.transforms.misc import Compose

data = Data(adata_inf, full_split_name="test")
data.append(Data(adata_ref, full_split_name="ref"), join="outer", label_batch=True)
transforms=Compose(
            CellTopicProfile(ct_select="auto", ct_key="cellType", batch_key=None, split_name="ref", method="mean"),
            CellGiottoTopicProfile(ct_select="auto", ct_key="cellType", split_name="ref",detection_threshold=-1),
            CellGiottoTopicProfile(ct_select="auto", ct_key="cellType", split_name="ref",detection_threshold=0,out="CellGiottoDetectionTopicProfile"),
            CellTypeNums(ct_select="auto", ct_key="cellType", split_name="ref"),
            FilterGenesMarkerGini()
)
transforms(data=data)


/home/zyxing/dance/dance/utils/matrix.py:135: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in numba.prange(n):
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
[INFO][2023-11-12 21:02:44,634][dance][__call__] Found 6 marker genes for cell type 'cell A'
[INFO][2023-11-12 21:02:44,649][dance][__call__] Found 8 marker genes for cell type 'cell B'
[INFO][2023-11-12 21:02:44,665][dance][__call__] Found 11 marker genes for cell type 'cell C'
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning:

In [8]:
data.data.uns["FilterGenesMarkerGini"].rename(columns={"ans_score":"comb_score","ans_rank":"comb_rank","gene_rank_score":"expression_rank","gene_detection_rank_score":"detection_rank"})

,comb_score,comb_rank,expression,detection,expression_gini,detection_gini,expression_rank,detection_rank,gene_name,cellType
2,0.073529,2,0.753649,1.000000,0.500000,0.147059,1.0,1.0,Vgf,cell A
3,0.041391,4,0.444901,0.888889,0.500000,0.082781,1.0,1.0,Lingo1,cell A
4,0.008389,7,0.462447,0.777778,0.500000,0.016779,1.0,1.0,Slc17a7,cell A
6,0.025000,5,0.526549,0.777778,0.500000,0.050000,1.0,1.0,Atp1a1,cell A
15,0.165094,1,0.470194,0.444444,0.500000,0.330189,1.0,1.0,Slc32a1,cell A
19,0.043893,3,0.345754,0.777778,0.500000,0.087786,1.0,1.0,Cplx1,cell A
0,0.100000,3,0.507656,0.500000,0.500000,0.200000,1.0,1.0,Cldn5,cell B
7,0.138889,1,0.967385,1.000000,0.500000,0.277778,1.0,1.0,Scg3,cell B
8,0.075581,6,0.352764,0.666667,0.500000,0.151163,1.0,1.0,Myh9,cell B
10,0.118421,2,0.837760,1.000000,0.500000,0.236842,1.0,1.0,Pygb,cell B


In [9]:
pd.read_csv("./spatial_DWLS_data/ref.csv")

,Unnamed: 0,genes,cluster,expression,expression_gini,detection,detection_gini,expression_rank,detection_rank,comb_score,comb_rank
0,1,Slc32a1,1,0.473484,0.5,0.444444,0.330189,1,1,0.165094,1
1,2,Vgf,1,0.748547,0.5,1.000000,0.147059,1,1,0.073529,2
2,3,Lingo1,1,0.444323,0.5,0.888889,0.082781,1,1,0.041391,3
3,4,Cplx1,1,0.323964,0.5,0.777778,0.050000,1,1,0.025000,4
4,5,Atp1a1,1,0.525773,0.5,0.777778,0.050000,1,1,0.025000,5
5,6,Scg3,2,0.970940,0.5,1.000000,0.277778,1,1,0.138889,1
6,7,Pygb,2,0.846047,0.5,1.000000,0.236842,1,1,0.118421,2
7,8,Col4a2,2,0.720211,0.5,0.833333,0.200000,1,1,0.100000,3
8,9,Sdc4,2,0.625966,0.5,0.833333,0.200000,1,1,0.100000,4
9,10,Mtss1l,2,0.600056,0.5,0.833333,0.125000,1,1,0.062500,7
